<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/T5/other_T5_evals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 726, done.
remote: Counting objects: 100% (460/460), done.
remote: Compressing objects: 100% (296/296), done.
remote: Total 726 (delta 222), reused 276 (delta 143), pack-reused 266
Receiving objects: 100% (726/726), 243.58 MiB | 17.28 MiB/s, done.
Resolving deltas: 100% (335/335), done.
Updating files: 100% (59/59), done.


In [2]:
!unzip '/content/RuLawSimplification/corpus/the_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
  inflating: the_corpus.csv          


In [3]:

import pandas as pd

df = pd.read_csv('the_corpus.csv')

In [4]:
df.columns

Index(['Название документа', 'Ссылка', 'Текст', 'Комментарий РГ', 'Date'], dtype='object')

In [5]:
df = df.drop(columns=['Название документа', 'Ссылка','Date'])

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42, test_size=0.05)

In [7]:
train.columns = ["text", "commentary"]
test.columns = ["text", "commentary"]

In [29]:
!pip install transformers rouge_score evaluate datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from transformers import pipeline

checkpoint = 'IlyaGusev/rut5_base_sum_gazeta'
summarizer = pipeline('summarization', model=checkpoint, tokenizer=checkpoint)

In [10]:
from tqdm import tqdm


predicted = []

for text in tqdm(test['text']):
    if len(text) > 3000:
      text = text[:3000]
      
    summarized = summarizer([text])[0]
    predicted.append(summarized['summary_text'])

100%|██████████| 102/102 [51:00<00:00, 30.00s/it]


In [35]:
from datasets import load_metric

rouge = load_metric("rouge")
result = rouge.compute(references=test['commentary'].to_list(), predictions=[str(p) for p in predicted])
result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
print(result)

{'rouge1': 9.354049661499186, 'rouge2': 2.3944309956796794, 'rougeL': 9.224577068785303, 'rougeLsum': 9.398069432067333}


In [12]:
!pip install sacrebleu sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=dc7f89048423349d6299acb053c8f934c58443b0345b42dd0aff45b7a826bd08
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [33]:
sari = evaluate.load("sari")

real = [list(orig) for orig in test['text']]


sari.compute(sources=test['text'].to_list(),  ## original
             references=[[ref] for ref in test['commentary'].to_list()], ## the comment from RG
             predictions=[str(p) for p in predicted]  ## model's version
             )

{'sari': 35.51942786467448}

In [14]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.5 MB/s eta 0:00:00


In [15]:
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=[str(p) for p in predicted], 
                            references=[[ref] for ref in test['commentary'].to_list()], lang='ru')
     

In [16]:
from statistics import mean
print('Precision: ', mean(results['precision']),
                          'F1:', mean(results['f1']))

Precision:  0.673158294429966 F1: 0.6332572651844398


In [17]:
import nltk

nltk.download('punkt')
## GUNNING FOG READABILITY INDEX

def gunning_fog(text):
  text = str(text)

  tokens = nltk.word_tokenize(text)
  tokens_length = len(tokens)
  sent_length = len(nltk.sent_tokenize(text))
  complex_tokens_len = len([tok for tok in tokens if len(tok) > 6])
  formule = 0.4 * (tokens_length / sent_length) + 100 * (complex_tokens_len / tokens_length)
  return formule

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
indexes = []

## original

for t in test['text'].to_list():
  indexes.append(gunning_fog(t))

print('Original text readability:', mean(indexes))


indexes = []

## simplified by model

for pred in predicted:
  indexes.append(gunning_fog(pred))

print('Summarized text readability:', mean(indexes))

Original text readability: 53.29907835008921
Summarized text readability: 62.706282784757974


In [19]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from razdel import sentenize, tokenize
import numpy as np


def all_syl_number(string):
    num_vowels = 0

    for char in string.lower():
        if char in "яыуаеиоюэ":
            num_vowels = num_vowels+1
       
    return num_vowels

def total_words(string):
    return len(list(tokenize(string)))

def total_sents(string):
    return len(list(sentenize(string)))
    
def fkgl(string):
    n_of_sents = total_sents(string)
    n_of_words = total_words(string)
    n_of_syls = all_syl_number(string)
    fkgl = (0.39 * (n_of_words/n_of_sents) + 11.8 * (n_of_syls/n_of_words)) - 15.39
    return float(fkgl)

# FKGL
print('FKGL')

indexes = []

## original

for t in test['text'].to_list():
  indexes.append(fkgl(t))

print('Original text readability:', np.mean(indexes))

indexes = []

## simplified by model

for pred in predicted:
  indexes.append(fkgl(pred))

print('Summarized text readability:', np.mean(indexes))

FKGL
Original text readability: 26.582805927512325
Summarized text readability: 29.049184920070907


In [21]:
import pandas as pd

df = pd.DataFrame({'Текст': test['text'],
                   'Комментарий РГ': predicted})

In [22]:
!unzip '/content/russian.zip'

unzip:  cannot find or open /content/russian.zip, /content/russian.zip.zip or /content/russian.zip.ZIP.


In [23]:

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import gensim



def topicmodelling(target, df):
    NUM_TOPICS = 10
    N_FIRST = 10 # сколько первых слов на тему выводить
    MIN_DF = 3 # минимальное число вхождений слова в документ, чтобы мы его учитывали (включали в вектор документа)
    MAX_DF = 0.99 # в каком проценте документов должно присутствовать слово, чтобы оно не рассматривалось

    df[target] = df[target].astype(str)
    stop_words = open("russian", 'r', encoding='utf-8').read()

    count_vectorizer = CountVectorizer(min_df=MIN_DF, max_df=MAX_DF, stop_words = [w for w in stop_words.split('\n')])
    data_count_vectorized = count_vectorizer.fit_transform(df['Текст'])

    lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='batch', batch_size=512, random_state = 2)
    lda_model.fit_transform(data_count_vectorized);

    temi_slova = []
    all_ = []

    for idx, topic in enumerate(lda_model.components_):
        all_.append([(count_vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-N_FIRST - 1:-1]])


    for i,topic in enumerate(all_):
        for para in topic:
            temi_slova.append(para[0])

    dic = {}

    for theme_id, topic in enumerate(all_):
        dic[theme_id] = []
        for para in topic:
            dic[theme_id].append(para[0])


    topics_10_words = []

    for topic_id in dic:
        topic = ' '.join(dic[topic_id][:10])
        topics_10_words.append(topic)

    with open(f'{target}_10.txt', 'w') as w:
        for t in topics_10_words:
            w.write(str(t))
            w.write('\n')

In [24]:
topicmodelling('Текст', df)
topicmodelling('Комментарий РГ', df)

FileNotFoundError: ignored

In [ ]:
with open('/content/Текст_10.txt', 'r') as f1:
    with open('/content/Комментарий РГ_10.txt', 'r') as f2: 
        first_orig = set([word for metka in f1.read().split('\n') for word in metka.split(' ')])
        first_comm = set([word for metka in f2.read().split('\n') for word in metka.split(' ')])

In [ ]:
n = 0
total = len(first_comm)
first_comm = set(first_comm)
first_orig = set(first_orig)


for label in first_comm:
    if label in first_orig:
        n += 1
        
print(n, '/', total)
print(n * 100 / total, '%')